### Phase 1: Preprocessing

In [1]:
# # =====================================
# # VIETNAMESE & ENGLISH TEXT EXTRACTION
# # GUARANTEED 4723 SAMPLES OUTPUT
# # =====================================

# import warnings
# warnings.filterwarnings('ignore')
# import os
# os.environ['TOKENIZERS_PARALLELISM'] = 'false'
# os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# !pip install -q transformers>=4.52.0 accelerate>=1.0.0
# !pip install -q av librosa easyocr datasets pandas scipy soundfile resampy

# import av
# import torch
# import numpy as np
# import pathlib
# import gc
# import easyocr
# import pandas as pd
# import time
# import random
# import re
# from transformers import pipeline
# from PIL import Image
# import librosa

# # Suppress logging
# import transformers
# transformers.logging.set_verbosity_error()
# import logging
# logging.getLogger().setLevel(logging.ERROR)

# device = "cuda" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# print(f"COMPLETE VI+EN EXTRACTION - Device: {device}")

# class Config:
#     # Dataset paths
#     dataset_train_path = "/kaggle/input/tikharm-dataset/Dataset/train"
#     dataset_val_path = "/kaggle/input/tikharm-dataset/Dataset/val"
#     dataset_test_path = "/kaggle/input/tikharm-dataset/Dataset/test"
#     extra_dataset_path = "/kaggle/working/extra_tikharm"
    
#     output_file = "./tikharm_vi_en_complete.csv"
#     extra_train_ratio = 0.85
    
#     # Processing settings
#     testing_mode = False
#     samples_per_class = None
    
#     # Audio settings - 60s limit
#     max_audio_duration = 60
#     min_audio_duration = 0.5
#     target_sample_rate = 16000
#     audio_chunk_size = 30
    
#     # OCR settings
#     enable_ocr = True
#     ocr_frames = 2
#     ocr_positions = [0.3, 0.7]
#     ocr_max_size = 640
    
#     # Processing
#     cleanup_interval = 15
#     progress_interval = 25
    
#     # ENSURE COMPLETE PROCESSING
#     save_all_samples = True  # Save all samples regardless of text extraction success
#     continue_on_error = True  # Continue processing even if individual videos fail

# config = Config()

# class AdvancedSpamFilter:
#     """Simplified spam filter - only removes clear spam/bias, keeps all real content"""
    
#     def __init__(self):
#         # ONLY CLEAR SPAM/BIAS PATTERNS - very specific
#         self.spam_patterns = [
#             # Vietnamese subscription calls - all variations
#             re.compile(r'.*hãy\s+(đăng\s*ký|subscribe).*kênh.*', re.IGNORECASE),
#             re.compile(r'.*(đăng\s*ký|subscribe).*kênh.*', re.IGNORECASE),
#             re.compile(r'.*subscribe.*cho.*kênh.*', re.IGNORECASE),
#             re.compile(r'.*theo\s*dõi.*kênh.*', re.IGNORECASE),
#             re.compile(r'.*nhấn.*đăng\s*ký.*kênh.*', re.IGNORECASE),
#             re.compile(r'.*bấm.*đăng\s*ký.*kênh.*', re.IGNORECASE),
            
#             # English subscription calls - MUST contain channel/subscribe together
#             re.compile(r'.*please\s+(subscribe|follow).*channel.*', re.IGNORECASE),
#             re.compile(r'.*(subscribe|follow).*channel.*please.*', re.IGNORECASE),
#             re.compile(r'.*hit.*subscribe.*channel.*', re.IGNORECASE),
#             re.compile(r'.*click.*subscribe.*channel.*', re.IGNORECASE),
            
#             # Vietnamese engagement spam - must be complete promotional phrases
#             re.compile(r'.*bấm\s+(like|chuông).*kênh.*', re.IGNORECASE),
#             re.compile(r'.*bấm.*chuông.*thông.*báo.*kênh.*', re.IGNORECASE),
#             re.compile(r'.*để\s+không\s+bỏ\s+lỡ.*video.*', re.IGNORECASE),
#             re.compile(r'.*(like|chia\s*sẻ).*video.*kênh.*', re.IGNORECASE),
            
#             # English engagement spam - must be complete promotional phrases
#             re.compile(r'.*(like|share).*video.*channel.*', re.IGNORECASE),
#             re.compile(r'.*hit.*notification.*bell.*', re.IGNORECASE),
#             re.compile(r'.*don\'t\s+forget.*subscribe.*', re.IGNORECASE),
#             re.compile(r'.*smash.*like.*subscribe.*', re.IGNORECASE),
            
#             # Brand/Channel specific spam
#             re.compile(r'.*ghiền\s+mì\s+gõ.*kênh.*', re.IGNORECASE),
            
#             # Social media platform calls - must be promotional
#             re.compile(r'.*(follow|theo\s*dõi).*(facebook|instagram|tiktok).*page.*', re.IGNORECASE),
            
#             # Generic promotional calls - must have multiple elements
#             re.compile(r'.*subscribe.*and.*like.*channel.*', re.IGNORECASE),
#             re.compile(r'.*like.*and.*subscribe.*channel.*', re.IGNORECASE),
#             re.compile(r'.*support.*our.*channel.*', re.IGNORECASE),
#             re.compile(r'.*ủng\s*hộ.*kênh.*của.*', re.IGNORECASE),
            
#             # Pure special characters only
#             re.compile(r'^[@#$%^&*()_+=\[\]{}|\\:";\'<>?,./\s]+$'),
            
#             # Pure numbers only  
#             re.compile(r'^[0-9\s\-\.]+$'),
#         ]
        
#         # ONLY EXTREME GIBBERISH - very conservative
#         self.extreme_gibberish_patterns = [
#             # 10+ consecutive repeated characters (very conservative)
#             re.compile(r'(.)\1{9,}'),
            
#             # 15+ consecutive Vietnamese diacritics (extremely rare)
#             re.compile(r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]{15,}'),
#         ]
    
#     def is_spam(self, text):
#         """Only detect clear spam/bias patterns"""
#         if not text or len(text.strip()) < 1:
#             return True
        
#         text_clean = text.lower().strip()
        
#         # Check for clear spam patterns
#         for pattern in self.spam_patterns:
#             if pattern.search(text_clean):
#                 return True
        
#         # Check for extreme gibberish only
#         for pattern in self.extreme_gibberish_patterns:
#             if pattern.search(text):
#                 return True
        
#         return False
    
#     def clean_repetition(self, text):
#         """Very conservative repetition cleaning"""
#         if not text:
#             return ""
        
#         # Only remove very excessive repetition (8+ times)
#         cleaned = re.sub(r'\b(\w+)(\s+\1){7,}\b', r'\1', text)
        
#         # Clean excessive spaces only
#         cleaned = re.sub(r'\s{8,}', ' ', cleaned)  # 8+ spaces -> 1
#         cleaned = cleaned.strip()
        
#         return cleaned
    
#     def process_text(self, text):
#         """Minimal processing - only remove clear spam"""
#         if not text:
#             return ""
        
#         # Basic cleaning
#         cleaned = text.strip()
#         cleaned = re.sub(r'\s+', ' ', cleaned)  # Multiple spaces to single
        
#         # Clean repetition very conservatively first
#         cleaned = self.clean_repetition(cleaned)
        
#         # Check for spam only AFTER cleaning
#         if self.is_spam(cleaned):
#             return ""
        
#         # NO minimum length requirement - keep all non-spam text
#         return cleaned

# class WhisperHallucinationFilter:
#     """Filter để loại bỏ Whisper hallucination patterns"""
    
#     def __init__(self):
#         # Common Whisper hallucination patterns
#         self.hallucination_patterns = [
#             # Exact repetitive phrases - very specific
#             re.compile(r'^thank\s+you\.?$', re.IGNORECASE),
#             re.compile(r'^oh\s+thank\s+you\.?$', re.IGNORECASE),
#             re.compile(r'^thank\s+you\.?\s+so\s+so\.?$', re.IGNORECASE),
#             re.compile(r'^thank\s+you\.?\s+you\.?$', re.IGNORECASE),
            
#             # Single word hallucinations
#             re.compile(r'^oh\.?$', re.IGNORECASE),
#             re.compile(r'^you\.?$', re.IGNORECASE),
#             re.compile(r'^bye\.?$', re.IGNORECASE),
#             re.compile(r'^hello\.?$', re.IGNORECASE),
#             re.compile(r'^hi\.?$', re.IGNORECASE),
            
#             # Repetitive short phrases
#             re.compile(r'^(oh\s+){2,}.*', re.IGNORECASE),
#             re.compile(r'^(you\s+){2,}.*', re.IGNORECASE),
#             re.compile(r'^(hey\s+){2,}.*', re.IGNORECASE),
            
#             # Very generic phrases that appear too often with low confidence
#             re.compile(r'^i\s+just\s+wanna\s+be\s+with\s+you\.?$', re.IGNORECASE),
#             re.compile(r'^everybody\s+make\s+some\s+noise\.?$', re.IGNORECASE),
#             re.compile(r'^everybody\s+make\s+some\s+noise\s+foreign\.?$', re.IGNORECASE),
            
#             # Very short generic greetings without context
#             re.compile(r'^(oh\s+)?(bye|hello|hi)\.?$', re.IGNORECASE),
#         ]
        
#         # Confidence-based filtering for very short phrases
#         self.low_confidence_short_phrases = [
#             'thank you', 'oh', 'you', 'bye', 'hello', 'hi', 'hey'
#         ]
        
#         # Phrases that are OK if they're part of longer sentences
#         self.context_dependent_phrases = [
#             'thank you', 'oh', 'you', 'bye', 'hello', 'hi'
#         ]
    
#     def is_likely_hallucination(self, text):
#         """Check if text is likely Whisper hallucination"""
#         if not text or len(text.strip()) < 1:
#             return True
        
#         text_clean = text.strip().lower()
#         word_count = len(text_clean.split())
        
#         # Check exact hallucination patterns
#         for pattern in self.hallucination_patterns:
#             if pattern.match(text_clean):
#                 return True
        
#         # Check for very short generic phrases (1-2 words)
#         if word_count <= 2:
#             for phrase in self.low_confidence_short_phrases:
#                 # If the entire text is just this phrase (with minimal variation)
#                 if phrase in text_clean and len(text_clean) <= len(phrase) + 3:
#                     return True
        
#         # Check for standalone context-dependent phrases
#         if word_count == 1:
#             if text_clean.rstrip('.!?') in ['oh', 'you', 'bye', 'hi', 'hey']:
#                 return True
        
#         return False
    
#     def filter_text(self, text):
#         """Filter out hallucinations but keep real content"""
#         if not text:
#             return ""
        
#         if self.is_likely_hallucination(text):
#             return ""
        
#         return text

# class FastLanguageDetector:
#     """Fast language detection - only Vietnamese and English"""
    
#     @staticmethod
#     def detect_language(text):
#         if not text or len(text.strip()) < 1:
#             return "other"
        
#         text_lower = text.lower()
        
#         vi_chars = len(re.findall(r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', text_lower))
#         vi_words = len(re.findall(r'\b(và|của|trong|với|từ|để|này|đó|có|là|được|một|tôi|bạn|anh|em|không|rất|cảm|ơn|tình|yêu|xin|chào|cám|chúc|mừng|được|làm|đi|về|gì|như|nào|khi|nếu|bây|giờ|hôm|nay|ngày|mai)\b', text_lower))
        
#         en_words = len(re.findall(r'\b(the|and|for|are|but|not|you|all|can|this|that|with|from|they|have|love|see|now|time|thank|hello|yes|no|good|great|nice|well|very|much|more|come|go|know|want|need|help|please|sorry|ok|okay|what|when|where|why|how|who|will|would|could|should|may|might|must|do|does|did|am|is|was|were|been|being|has|had|having|get|got|give|take|make|made|say|said|tell|told|ask|think|feel|look|seems|try|work|play|use|used|like|want|need)\b', text_lower))
        
#         other_asian = len(re.findall(r'[\u0E00-\u0E7F\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]', text))
        
#         # Reject other Asian languages
#         if other_asian > len(text) * 0.1:
#             return "other"
        
#         # Scoring system
#         vi_score = vi_chars * 3 + vi_words * 4
#         en_score = en_words * 3
        
#         # For very short text (1-3 words), be more specific
#         word_count = len(text.split())
#         if word_count <= 3:
#             # If has Vietnamese characters or clear Vietnamese words
#             if vi_chars > 0:
#                 return "vietnamese"
#             elif vi_words > 0:
#                 return "vietnamese"
#             # If has clear English words
#             elif en_words > 0:
#                 return "english"
#             # If it's basic English words that might not be in the regex
#             elif any(word in text_lower for word in ['thank', 'hello', 'yes', 'no', 'ok', 'good', 'bad', 'nice', 'great', 'wow', 'ah', 'um', 'hmm']):
#                 return "english"
#             else:
#                 return "other"
        
#         # For longer text, use scores
#         if vi_score > en_score and vi_score > 0:
#             return "vietnamese"
#         elif en_score > 0:
#             return "english"
#         elif vi_chars > 0:  # Has Vietnamese diacritics
#             return "vietnamese"
#         else:
#             return "other"
    
#     @staticmethod
#     def is_target_language(text):
#         detected = FastLanguageDetector.detect_language(text)
#         # ONLY accept Vietnamese and English
#         return detected in ["vietnamese", "english"]

# class FastOCRExtractor:
#     """Fast OCR extraction with error handling"""
    
#     def __init__(self):
#         print("Loading Fast EasyOCR...")
#         try:
#             self.reader = easyocr.Reader(['vi', 'en'], gpu=torch.cuda.is_available(), verbose=False)
#             print("✓ OCR Ready")
#             self.available = True
#         except Exception as e:
#             print(f"✗ OCR Failed: {e}")
#             self.reader = None
#             self.available = False
        
#         self.spam_filter = AdvancedSpamFilter()
    
#     def extract_text_safe(self, video_path):
#         """Safe OCR extraction with comprehensive error handling"""
#         if not self.available or not self.reader:
#             return ""
        
#         try:
#             container = av.open(str(video_path))
#             if not container.streams.video:
#                 container.close()
#                 return ""
            
#             video_stream = container.streams.video[0]
#             total_frames = max(video_stream.frames or 1000, 1000)
#             frame_positions = [int(total_frames * pos) for pos in config.ocr_positions]
            
#             all_texts = []
#             current_frame = 0
#             extracted = 0
            
#             for frame in container.decode(video=0):
#                 if current_frame in frame_positions:
#                     try:
#                         frame_array = frame.to_ndarray(format="rgb24")
                        
#                         # Fast resize
#                         h, w = frame_array.shape[:2]
#                         if max(h, w) > config.ocr_max_size:
#                             scale = config.ocr_max_size / max(h, w)
#                             new_h, new_w = int(h * scale), int(w * scale)
#                             img = Image.fromarray(frame_array)
#                             img = img.resize((new_w, new_h), Image.LANCZOS)
#                             frame_array = np.array(img)
                        
#                         # Safe OCR call
#                         results = self.reader.readtext(
#                             frame_array, 
#                             detail=0, 
#                             width_ths=0.7, 
#                             height_ths=0.7
#                         )
                        
#                         for text in results:
#                             if isinstance(text, str) and text.strip():
#                                 processed = self.spam_filter.process_text(text.strip())
#                                 if processed:
#                                     # ONLY accept Vietnamese and English
#                                     if FastLanguageDetector.is_target_language(processed):
#                                         all_texts.append(processed)
                        
#                         extracted += 1
#                     except Exception as ocr_error:
#                         # Continue on individual frame errors
#                         continue
                
#                 current_frame += 1
#                 if extracted >= config.ocr_frames:
#                     break
            
#             container.close()
            
#             if all_texts:
#                 unique_texts = list(dict.fromkeys(all_texts))
#                 combined = ' '.join(unique_texts)
#                 final = self.spam_filter.process_text(combined)
#                 # Final check for target languages
#                 if final and FastLanguageDetector.is_target_language(final):
#                     return final
            
#             return ""
            
#         except Exception as e:
#             # Return empty string on any error, don't crash
#             return ""

# class FastAudioExtractor:
#     """Fast audio extraction with comprehensive error handling"""
    
#     def __init__(self):
#         print("Loading Fast Whisper...")
#         try:
#             self.pipe = pipeline(
#                 "automatic-speech-recognition",
#                 model="openai/whisper-large-v3-turbo",
#                 torch_dtype=torch_dtype,
#                 device=device,
#                 return_timestamps=False,
#                 chunk_length_s=config.audio_chunk_size,
#                 stride_length_s=5
#             )
#             print("✓ Whisper Ready")
#             self.available = True
#         except Exception as e:
#             print(f"✗ Whisper Failed: {e}")
#             self.pipe = None
#             self.available = False
        
#         self.spam_filter = AdvancedSpamFilter()
#         self.hallucination_filter = WhisperHallucinationFilter()  # Thêm filter mới
    
#     def get_video_duration_safe(self, video_path):
#         """Safe video duration check"""
#         try:
#             container = av.open(str(video_path))
#             if container.streams.video:
#                 duration = float(container.duration) / av.time_base if container.duration else 0
#                 container.close()
#                 return min(duration, config.max_audio_duration)
#             container.close()
#             return 0
#         except:
#             return config.max_audio_duration
    
#     def extract_audio_safe(self, video_path):
#         """Safe audio extraction"""
#         try:
#             video_duration = self.get_video_duration_safe(video_path)
#             duration_to_load = min(video_duration, config.max_audio_duration)
            
#             if duration_to_load < config.min_audio_duration:
#                 return None, 0
            
#             audio, _ = librosa.load(
#                 str(video_path),
#                 sr=config.target_sample_rate,
#                 duration=duration_to_load,
#                 mono=True,
#                 res_type='kaiser_fast'
#             )
            
#             if len(audio) < config.min_audio_duration * config.target_sample_rate:
#                 return None, 0
            
#             rms = np.sqrt(np.mean(audio**2))
#             if rms < 1e-6:
#                 return None, 0
            
#             max_val = np.abs(audio).max()
#             if max_val > 0:
#                 audio = audio / max_val * 0.8
            
#             actual_duration = len(audio) / config.target_sample_rate
            
#             return audio.astype(np.float32), actual_duration
#         except:
#             return None, 0
    
#     def transcribe_text_safe(self, video_path):
#         """Safe audio transcription with comprehensive error handling"""
#         if not self.available or not self.pipe:
#             return ""
        
#         try:
#             audio, duration = self.extract_audio_safe(video_path)
#             if audio is None:
#                 return ""
            
#             best_result = ""
            
#             # Try Vietnamese first, then English
#             for language in ["vi", "en"]:
#                 try:
#                     result = self.pipe(
#                         audio,
#                         generate_kwargs={
#                             "language": language,
#                             "task": "transcribe",
#                             "temperature": 0.0,
#                             "no_speech_threshold": 0.7,  # Tăng từ 0.6 để giảm hallucination
#                             "compression_ratio_threshold": 2.4,
#                             "logprob_threshold": -1.0,
#                             "no_repeat_ngram_size": 3,
#                             "repetition_penalty": 1.2  # Tăng từ 1.1
#                         }
#                     )
                    
#                     text = result.get('text', '').strip() if isinstance(result, dict) else str(result).strip()
                    
#                     if text:
#                         # Kiểm tra hallucination TRƯỚC khi xử lý
#                         if self.hallucination_filter.is_likely_hallucination(text):
#                             continue  # Skip hallucination
                        
#                         processed = self.spam_filter.process_text(text)
#                         if processed and len(processed) > len(best_result):
#                             detected = FastLanguageDetector.detect_language(processed)
#                             # ONLY accept Vietnamese and English
#                             if detected in ["vietnamese", "english"]:
#                                 # Final hallucination check
#                                 final_filtered = self.hallucination_filter.filter_text(processed)
#                                 if final_filtered:
#                                     best_result = final_filtered
#                                     # Prefer Vietnamese and longer text
#                                     if detected == "vietnamese" and len(final_filtered) > 5:
#                                         break
#                 except Exception as whisper_error:
#                     # Continue on individual language errors
#                     continue
            
#             # Final validation
#             if best_result and FastLanguageDetector.is_target_language(best_result):
#                 return best_result
#             else:
#                 return ""
            
#         except Exception as e:
#             # Return empty string on any error, don't crash
#             return ""

# class CompleteProcessor:
#     """Complete processor that guarantees all samples are processed"""
    
#     def __init__(self):
#         self.ocr_extractor = FastOCRExtractor()
#         self.audio_extractor = FastAudioExtractor()
        
#         # Comprehensive stats
#         self.total_discovered = 0
#         self.total_processed = 0
#         self.total_completed = 0
#         self.audio_success = 0
#         self.ocr_success = 0
#         self.combined_success = 0
#         self.vietnamese_count = 0
#         self.english_count = 0
#         self.processing_errors = 0
#         self.hallucination_filtered = 0  # Track filtered hallucinations
        
#         # Timing stats
#         self.audio_time = 0
#         self.ocr_time = 0
        
#         print(f"Processor initialized:")
#         print(f"  OCR available: {self.ocr_extractor.available}")
#         print(f"  Audio available: {self.audio_extractor.available}")
#         print(f"  Hallucination filter: Enabled")
    
#     def process_video_safe(self, video_record):
#         """Safe video processing that always returns a record"""
#         video_path = pathlib.Path(video_record['video_path'])
#         video_name = video_path.name
        
#         self.total_processed += 1
        
#         # Initialize with empty values
#         audio_text = ""
#         ocr_text = ""
#         processing_status = "completed"
#         error_message = ""
        
#         try:
#             # Extract audio with error handling
#             if self.audio_extractor.available:
#                 try:
#                     audio_start = time.time()
#                     audio_text = self.audio_extractor.transcribe_text_safe(video_path)
#                     self.audio_time += time.time() - audio_start
                    
#                     if audio_text:
#                         self.audio_success += 1
#                         lang = FastLanguageDetector.detect_language(audio_text)
#                         if lang == "vietnamese":
#                             self.vietnamese_count += 1
#                         elif lang == "english":
#                             self.english_count += 1
#                 except Exception as audio_error:
#                     audio_text = ""
#                     error_message += f"Audio error: {str(audio_error)[:50]}; "
            
#             # Extract OCR with error handling
#             if self.ocr_extractor.available:
#                 try:
#                     ocr_start = time.time()
#                     ocr_text = self.ocr_extractor.extract_text_safe(video_path)
#                     self.ocr_time += time.time() - ocr_start
                    
#                     if ocr_text:
#                         self.ocr_success += 1
#                 except Exception as ocr_error:
#                     ocr_text = ""
#                     error_message += f"OCR error: {str(ocr_error)[:50]}; "
            
#         except Exception as general_error:
#             processing_status = "error"
#             error_message += f"General error: {str(general_error)[:50]}; "
#             self.processing_errors += 1
        
#         # Combine results
#         parts = []
#         if audio_text:
#             parts.append(f"AUDIO: {audio_text}")
#         if ocr_text:
#             parts.append(f"OCR: {ocr_text}")
        
#         combined = " | ".join(parts) if parts else ""
        
#         if combined:
#             self.combined_success += 1
        
#         # ALWAYS update record - this ensures we keep all 4723 samples
#         video_record['ocr_text'] = ocr_text
#         video_record['audio_text'] = audio_text
#         video_record['combined_text'] = combined
#         video_record['processing_status'] = processing_status
#         video_record['error_message'] = error_message.strip()
        
#         self.total_completed += 1
        
#         # Print result
#         if combined:
#             print(f"✓ {video_name}")
#             if audio_text:
#                 print(f"  Audio: {audio_text[:80]}{'...' if len(audio_text) > 80 else ''}")
#             if ocr_text:
#                 print(f"  OCR: {ocr_text[:80]}{'...' if len(ocr_text) > 80 else ''}")
#         elif processing_status == "error":
#             print(f"✗ {video_name} - Error: {error_message[:100]}")
#         else:
#             print(f"○ {video_name} - No valid text extracted")
        
#         return video_record
    
#     def get_comprehensive_stats(self):
#         return {
#             'total_discovered': self.total_discovered,
#             'total_processed': self.total_processed,
#             'total_completed': self.total_completed,
#             'audio_success': self.audio_success,
#             'ocr_success': self.ocr_success,
#             'combined_success': self.combined_success,
#             'vietnamese_count': self.vietnamese_count,
#             'english_count': self.english_count,
#             'processing_errors': self.processing_errors,
#             'hallucination_filtered': self.hallucination_filtered,
#             'avg_audio_time': self.audio_time / max(self.total_processed, 1),
#             'avg_ocr_time': self.ocr_time / max(self.total_processed, 1)
#         }
    
#     def cleanup(self):
#         gc.collect()
#         if torch.cuda.is_available():
#             torch.cuda.empty_cache()

# def discover_all_videos():
#     """Complete video discovery with guaranteed count"""
#     print("Discovering ALL videos...")
    
#     records = []
#     datasets = {
#         'train': config.dataset_train_path,
#         'val': config.dataset_val_path,
#         'test': config.dataset_test_path
#     }
    
#     # Check extra dataset
#     if pathlib.Path(config.extra_dataset_path).exists():
#         datasets['extra'] = config.extra_dataset_path
#         print(f"✓ Extra dataset found")
#     else:
#         print(f"✗ Extra dataset not found at {config.extra_dataset_path}")
    
#     for name, path in datasets.items():
#         if not pathlib.Path(path).exists():
#             print(f"✗ Dataset not found: {path}")
#             continue
        
#         videos = list(pathlib.Path(path).glob("**/*.mp4"))
#         print(f"{name}: {len(videos)} videos discovered")
        
#         # Group by class
#         by_class = {}
#         for video in videos:
#             class_name = video.parent.name
#             if class_name not in by_class:
#                 by_class[class_name] = []
#             by_class[class_name].append(video)
        
#         print(f"  Classes in {name}: {list(by_class.keys())}")
        
#         for class_name, class_videos in by_class.items():
#             # Testing mode (if enabled)
#             if config.testing_mode and config.samples_per_class:
#                 random.shuffle(class_videos)
#                 class_videos = class_videos[:config.samples_per_class]
#                 print(f"  {class_name}: {len(class_videos)} videos (limited for testing)")
            
#             for video in class_videos:
#                 # Split assignment
#                 if name == 'extra':
#                     split = 'train' if random.random() < config.extra_train_ratio else 'val'
#                 else:
#                     split = name
                
#                 records.append({
#                     'video_path': str(video),
#                     'video_name': video.name,
#                     'class_name': class_name,
#                     'original_dir': name,
#                     'split': split,
#                     'ocr_text': "",
#                     'audio_text': "",
#                     'combined_text': "",
#                     'processing_status': "pending",
#                     'error_message': ""
#                 })
    
#     print(f"\n{'='*50}")
#     print(f"TOTAL VIDEOS DISCOVERED: {len(records)}")
#     print(f"{'='*50}")
    
#     # Show distribution
#     for dataset in ['train', 'val', 'test', 'extra']:
#         dataset_count = len([r for r in records if r['original_dir'] == dataset])
#         if dataset_count > 0:
#             print(f"  {dataset}: {dataset_count} videos")
    
#     split_counts = {}
#     for record in records:
#         split = record['split']
#         split_counts[split] = split_counts.get(split, 0) + 1
    
#     print(f"\nSplit distribution:")
#     for split, count in split_counts.items():
#         print(f"  {split}: {count} videos")
    
#     return records

# def run_complete_extraction():
#     """Complete extraction ensuring all 4723 samples are processed"""
#     print("=" * 60)
#     print("COMPLETE VI+EN EXTRACTION - ALL 4723 SAMPLES GUARANTEED")
#     print("=" * 60)
    
#     # Discover all videos
#     all_videos = discover_all_videos()
    
#     if not all_videos:
#         print("ERROR: No videos found!")
#         return None
    
#     total_videos = len(all_videos)
#     print(f"\nStarting processing of {total_videos} videos...")
#     print(f"Target: Process ALL {total_videos} samples regardless of success/failure")
#     print(f"Features: Spam filter + Hallucination filter + Vi/En detection")
    
#     # Initialize processor
#     processor = CompleteProcessor()
#     processor.total_discovered = total_videos
    
#     start_time = time.time()
#     successful_processing = 0
    
#     try:
#         for i, video in enumerate(all_videos):
#             # Process video (always returns a record)
#             processed_video = processor.process_video_safe(video)
            
#             # Update the original record
#             all_videos[i] = processed_video
            
#             if processed_video['combined_text']:
#                 successful_processing += 1
            
#             # Progress updates
#             if (i + 1) % config.progress_interval == 0 or (i + 1) == total_videos:
#                 elapsed = time.time() - start_time
#                 speed = (i + 1) / (elapsed / 60) if elapsed > 0 else 0
#                 eta = (total_videos - i - 1) / max(speed, 1) if speed > 0 else 0
                
#                 stats = processor.get_comprehensive_stats()
                
#                 print(f"\n{'='*60}")
#                 print(f"Progress: {i + 1}/{total_videos} ({(i + 1)/total_videos*100:.1f}%)")
#                 print(f"Speed: {speed:.1f} videos/min")
#                 if eta > 0:
#                     print(f"ETA: {eta:.1f} minutes")
#                 print(f"")
#                 print(f"Results so far:")
#                 print(f"  Processed: {stats['total_processed']}/{total_videos}")
#                 print(f"  Completed: {stats['total_completed']}/{total_videos}")
#                 print(f"  Audio success: {stats['audio_success']} ({stats['audio_success']/(i+1)*100:.1f}%)")
#                 print(f"  OCR success: {stats['ocr_success']} ({stats['ocr_success']/(i+1)*100:.1f}%)")
#                 print(f"  Combined success: {stats['combined_success']} ({stats['combined_success']/(i+1)*100:.1f}%)")
#                 print(f"  Processing errors: {stats['processing_errors']}")
#                 print(f"  Languages - Vi: {stats['vietnamese_count']}, En: {stats['english_count']}")
#                 print(f"  Avg time - Audio: {stats['avg_audio_time']:.1f}s, OCR: {stats['avg_ocr_time']:.1f}s")
#                 print(f"{'='*60}")
            
#             # Memory cleanup
#             if (i + 1) % config.cleanup_interval == 0:
#                 processor.cleanup()
    
#     except KeyboardInterrupt:
#         print(f"\n⚠️  INTERRUPTED after {processor.total_processed} videos")
#         print(f"Saving partial results...")
#     except Exception as e:
#         print(f"\n❌ FATAL ERROR after {processor.total_processed} videos: {e}")
#         print(f"Saving partial results...")
#         import traceback
#         traceback.print_exc()
    
#     finally:
#         processor.cleanup()
    
#     # Create final DataFrame - GUARANTEED to have all discovered videos
#     print(f"\nCreating final DataFrame...")
#     df = pd.DataFrame(all_videos)
    
#     # Verify we have all samples
#     final_count = len(df)
#     print(f"DataFrame created with {final_count} samples")
    
#     if final_count != total_videos:
#         print(f"⚠️  WARNING: Expected {total_videos} samples, got {final_count}")
#     else:
#         print(f"✅ SUCCESS: All {total_videos} samples preserved in DataFrame")
    
#     # Final statistics
#     processing_time = time.time() - start_time
#     stats = processor.get_comprehensive_stats()
    
#     # Count actual results
#     audio_total = (df['audio_text'].str.strip() != '').sum()
#     ocr_total = (df['ocr_text'].str.strip() != '').sum()
#     combined_total = (df['combined_text'].str.strip() != '').sum()
#     error_total = (df['processing_status'] == 'error').sum()
    
#     print(f"\n{'='*60}")
#     print("FINAL COMPLETE RESULTS")
#     print(f"{'='*60}")
#     print(f"Total samples in dataset: {final_count}")
#     print(f"Successfully processed: {stats['total_completed']}")
#     print(f"Audio extraction success: {audio_total} ({audio_total/final_count*100:.1f}%)")
#     print(f"OCR extraction success: {ocr_total} ({ocr_total/final_count*100:.1f}%)")
#     print(f"Combined text success: {combined_total} ({combined_total/final_count*100:.1f}%)")
#     print(f"Processing errors: {error_total} ({error_total/final_count*100:.1f}%)")
#     print(f"Empty results (no text): {final_count - combined_total} ({(final_count - combined_total)/final_count*100:.1f}%)")
#     print(f"")
#     print(f"Language distribution:")
#     print(f"  Vietnamese: {stats['vietnamese_count']}")
#     print(f"  English: {stats['english_count']}")
#     print(f"")
#     print(f"Processing time: {processing_time/60:.1f} minutes")
#     print(f"Average speed: {final_count/(processing_time/60):.1f} videos/minute")
#     print(f"Average per video: {processing_time/final_count:.1f} seconds")
    
#     # Dataset breakdown - COMPLETE
#     print(f"\nComplete dataset breakdown:")
#     for dataset in ['train', 'val', 'test', 'extra']:
#         dataset_df = df[df['original_dir'] == dataset]
#         if len(dataset_df) > 0:
#             with_text = (dataset_df['combined_text'].str.strip() != '').sum()
#             print(f"  {dataset}: {len(dataset_df)} videos, {with_text} with text ({with_text/len(dataset_df)*100:.1f}%)")
    
#     print(f"\nSplit breakdown:")
#     for split in ['train', 'val', 'test']:
#         split_df = df[df['split'] == split]
#         if len(split_df) > 0:
#             with_text = (split_df['combined_text'].str.strip() != '').sum()
#             print(f"  {split}: {len(split_df)} videos, {with_text} with text ({with_text/len(split_df)*100:.1f}%)")
    
#     # Save complete results
#     print(f"\nSaving complete dataset...")
#     df.to_csv(config.output_file, index=False)
#     print(f"✅ Complete dataset saved: {config.output_file}")
#     print(f"   Contains all {len(df)} samples with columns:")
#     print(f"   {list(df.columns)}")
    
#     # Show sample successful results
#     successful = df[df['combined_text'].str.strip() != ''].head(3)
#     if len(successful) > 0:
#         print(f"\nSample successful extractions:")
#         for _, row in successful.iterrows():
#             print(f"  📁 {row['video_name']} ({row['class_name']})")
#             if row['audio_text']:
#                 print(f"     🎵 Audio: {row['audio_text'][:80]}{'...' if len(row['audio_text']) > 80 else ''}")
#             if row['ocr_text']:
#                 print(f"     🖼️  OCR: {row['ocr_text'][:80]}{'...' if len(row['ocr_text']) > 80 else ''}")
    
#     # Show sample empty results
#     empty = df[df['combined_text'].str.strip() == ''].head(2)
#     if len(empty) > 0:
#         print(f"\nSample empty results (still preserved):")
#         for _, row in empty.iterrows():
#             print(f"  📁 {row['video_name']} ({row['class_name']}) - Status: {row['processing_status']}")
#             if row['error_message']:
#                 print(f"     ⚠️  Error: {row['error_message'][:60]}...")
    
#     # Show filtering effectiveness
#     if stats['hallucination_filtered'] > 0:
#         print(f"\n🛡️  Hallucination filter effectiveness:")
#         print(f"   Filtered out: {stats['hallucination_filtered']} potential hallucinations")
#         print(f"   Quality improvement: Reduced false positive extractions")
    
#     return df

# # Execute complete extraction
# if __name__ == "__main__":
#     try:
#         # CUDA optimizations
#         if torch.cuda.is_available():
#             torch.backends.cudnn.benchmark = True
#             torch.backends.cuda.matmul.allow_tf32 = True
        
#         # Seeds for reproducibility
#         random.seed(42)
#         np.random.seed(42)
#         torch.manual_seed(42)
        
#         # Run complete extraction
#         results = run_complete_extraction()
        
#         if results is not None:
#             print(f"\n🎉 COMPLETE SUCCESS!")
#             print(f"📊 All 4723 samples processed and saved")
#             print(f"🇻🇳🇺🇸 Vietnamese + English extraction completed")
#             print(f"🛡️  Spam + Hallucination filtering applied")
#             print(f"💾 Complete dataset ready for use")
#             print(f"")
#             print(f"Final verification:")
#             print(f"  Expected samples: 4723")
#             print(f"  Actual samples: {len(results)}")
#             print(f"  Match: {'✅ YES' if len(results) == 4723 else '❌ NO'}")
#         else:
#             print(f"\n❌ FAILED - No results generated")
    
#     except Exception as e:
#         print(f"\nFATAL ERROR: {e}")
#         import traceback
#         traceback.print_exc()
    
#     finally:
#         gc.collect()
#         if torch.cuda.is_available():
#             torch.cuda.empty_cache()
#         print("\nProcessing completed!")

## Phase 2: Training

In [2]:
# Install required packages first
!pip install -q transformers>=4.52.0 accelerate>=1.0.0 datasets>=3.0.0
!pip install -q av librosa soundfile resampy
!pip install -q scikit-learn matplotlib seaborn
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q pytorchvideo
!pip install -q evaluate
!pip install -q datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import os
import gc
import cv2
import warnings
from typing import Dict, List, Optional, Tuple, Union
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score
from dataclasses import dataclass
import time
import subprocess
import sys
from contextlib import contextmanager, redirect_stderr
import io

# Transformers components
from transformers import (
    AutoTokenizer, AutoModel, AutoImageProcessor,
    TimesformerForVideoClassification, Trainer, TrainingArguments, 
    EvalPrediction, PreTrainedModel, PretrainedConfig
)
from torch.utils.data import Dataset, DataLoader
import evaluate

# =============================================================================
# COMPLETE ERROR SUPPRESSION AND ENVIRONMENT SETUP
# =============================================================================

def setup_ultimate_silent_environment():
    """Setup the most comprehensive silent environment"""
    
    # Environment variables for complete silence
    silent_vars = {
        'OPENCV_LOG_LEVEL': 'SILENT',
        'OPENCV_VIDEOIO_DEBUG': '0',
        'OPENCV_FFMPEG_DEBUG': '0',
        'OPENCV_VIDEOWRITER_DEBUG': '0',
        'OPENCV_VIDEOCAPTURE_DEBUG': '0',
        'OPENCV_FFMPEG_LOGLEVEL': '-8',
        'OPENCV_AVFOUNDATION_SKIP_AUTH': '1',
        'FFREPORT': 'file=/dev/null:level=quiet',
        'WANDB_DISABLED': 'true',
        'TOKENIZERS_PARALLELISM': 'false',
        'PYTHONWARNINGS': 'ignore',
        'CUDA_LAUNCH_BLOCKING': '0'
    }
    
    for key, value in silent_vars.items():
        os.environ[key] = value
    
    # Configure OpenCV for complete silence
    try:
        cv2.setLogLevel(0)
        cv2.setUseOptimized(True)
    except:
        pass
    
    # Suppress all warnings
    warnings.filterwarnings('ignore')
    warnings.simplefilter('ignore')

@contextmanager
def ultimate_stderr_suppression():
    """Most advanced stderr suppression for C libraries"""
    old_stderr = None
    try:
        if hasattr(os, 'devnull'):
            old_stderr = sys.stderr
            sys.stderr = open(os.devnull, 'w')
            yield
        else:
            old_stderr = sys.stderr
            sys.stderr = io.StringIO()
            yield
    except:
        yield
    finally:
        if old_stderr is not None:
            try:
                if hasattr(sys.stderr, 'close'):
                    sys.stderr.close()
            except:
                pass
            sys.stderr = old_stderr

# Setup environment immediately
setup_ultimate_silent_environment()
print("=== SMART MULTIMODAL CLASSIFIER - ADAPTIVE VALIDATION (FIXED) ===")

def cleanup_memory():
    """Enhanced memory cleanup"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

cleanup_memory()

# =============================================================================
# 1. SMART VIDEO VALIDATION SYSTEM (SAME AS BEFORE)
# =============================================================================

class SmartVideoValidator:
    """Smart video validator with adaptive fallback strategies"""
    
    def __init__(self, strict_mode=False):
        self.strict_mode = strict_mode
        self.validation_cache = {}
        self.ffprobe_available = None
        self.stats = {
            'total': 0, 'valid': 0, 'invalid': 0,
            'not_found': 0, 'too_small': 0, 
            'ffprobe_failed': 0, 'opencv_failed': 0,
            'opencv_success': 0, 'ffprobe_success': 0
        }
        
        # Check FFprobe availability
        self._check_ffprobe_availability()
        
        print(f"📋 Validator initialized:")
        print(f"  FFprobe available: {self.ffprobe_available}")
        print(f"  Strict mode: {self.strict_mode}")
    
    def _check_ffprobe_availability(self):
        """Check if FFprobe is available on the system"""
        try:
            with ultimate_stderr_suppression():
                result = subprocess.run(['ffprobe', '-version'], 
                                      capture_output=True, 
                                      timeout=5)
            self.ffprobe_available = (result.returncode == 0)
        except:
            self.ffprobe_available = False
        
        if not self.ffprobe_available:
            print("⚠️ FFprobe not available - will use OpenCV-only validation")
    
    def is_video_valid(self, video_path: str) -> bool:
        """Smart video validation with adaptive strategies"""
        
        if video_path in self.validation_cache:
            return self.validation_cache[video_path]
        
        self.stats['total'] += 1
        is_valid = False
        
        try:
            # Step 1: Basic checks
            if not video_path or not os.path.exists(video_path):
                self.stats['not_found'] += 1
                self.validation_cache[video_path] = False
                return False
            
            # Step 2: File size check (more lenient)
            try:
                file_size = os.path.getsize(video_path)
                if file_size < 1000:  # Less than 1KB
                    self.stats['too_small'] += 1
                    self.validation_cache[video_path] = False
                    return False
            except:
                self.stats['too_small'] += 1
                self.validation_cache[video_path] = False
                return False
            
            # Step 3: Try FFprobe if available, otherwise use OpenCV
            if self.ffprobe_available:
                if self._validate_with_ffprobe(video_path):
                    is_valid = True
                    self.stats['ffprobe_success'] += 1
                else:
                    self.stats['ffprobe_failed'] += 1
                    # If FFprobe fails, try OpenCV as fallback
                    if self._validate_with_opencv(video_path):
                        is_valid = True
                        self.stats['opencv_success'] += 1
                    else:
                        self.stats['opencv_failed'] += 1
            else:
                # OpenCV-only validation
                if self._validate_with_opencv(video_path):
                    is_valid = True
                    self.stats['opencv_success'] += 1
                else:
                    self.stats['opencv_failed'] += 1
                
        except:
            self.stats['invalid'] += 1
        
        if not is_valid:
            self.stats['invalid'] += 1
        else:
            self.stats['valid'] += 1
        
        self.validation_cache[video_path] = is_valid
        return is_valid
    
    def _validate_with_ffprobe(self, video_path: str) -> bool:
        """FFprobe validation with lenient criteria"""
        try:
            cmd = [
                'ffprobe', '-v', 'quiet', '-select_streams', 'v:0',
                '-show_entries', 'stream=width,height,nb_frames',
                '-of', 'csv=p=0', video_path
            ]
            
            with ultimate_stderr_suppression():
                result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
            
            if result.returncode != 0:
                return False
            
            # Parse output
            output = result.stdout.strip()
            if not output:
                return False
            
            try:
                parts = output.split(',')
                if len(parts) >= 2:
                    width = int(parts[0]) if parts[0] else 0
                    height = int(parts[1]) if parts[1] else 0
                    
                    # Very lenient criteria
                    if width > 0 and height > 0:
                        return True
            except (ValueError, IndexError):
                pass
            
            return False
            
        except:
            return False
    
    def _validate_with_opencv(self, video_path: str) -> bool:
        """OpenCV validation with lenient criteria"""
        cap = None
        try:
            with ultimate_stderr_suppression():
                # Try different backends
                for backend in [cv2.CAP_FFMPEG, cv2.CAP_ANY]:
                    try:
                        cap = cv2.VideoCapture(video_path, backend)
                        if cap and cap.isOpened():
                            break
                    except:
                        continue
                
                if not cap or not cap.isOpened():
                    return False
                
                # Get basic properties
                frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                
                # Very lenient criteria - just check if we can get basic properties
                if width > 0 and height > 0:
                    # Try to read at least one frame
                    ret, frame = cap.read()
                    if ret and frame is not None and frame.size > 0:
                        return True
                
                return False
                
        except:
            return False
        finally:
            if cap:
                try:
                    cap.release()
                except:
                    pass
    
    def validate_dataset_sample(self, df: pd.DataFrame, sample_size: int = 100) -> pd.DataFrame:
        """Validate a sample first to check validation success rate"""
        print(f"🧪 Testing validation on {sample_size} samples...")
        
        sample_df = df.head(sample_size) if len(df) > sample_size else df
        valid_count = 0
        
        for idx, row in sample_df.iterrows():
            video_path = row['video_path']
            if self.is_video_valid(video_path):
                valid_count += 1
        
        success_rate = (valid_count / len(sample_df)) * 100
        print(f"📊 Sample validation results:")
        print(f"  Tested: {len(sample_df)} files")
        print(f"  Valid: {valid_count}")
        print(f"  Success rate: {success_rate:.1f}%")
        
        if success_rate < 10:
            print("⚠️ Very low validation success rate!")
            print("🔄 Switching to lenient OpenCV-only mode...")
            return self._validate_dataset_lenient(df)
        else:
            return self._validate_dataset_normal(df)
    
    def _validate_dataset_normal(self, df: pd.DataFrame) -> pd.DataFrame:
        """Normal validation process"""
        print("🔍 Performing normal video validation...")
        
        valid_indices = []
        total = len(df)
        
        for idx, row in df.iterrows():
            video_path = row['video_path']
            if self.is_video_valid(video_path):
                valid_indices.append(idx)
            
            if (idx + 1) % 500 == 0:
                print(f"  Progress: {idx + 1}/{total} files validated...")
        
        return df.iloc[valid_indices].reset_index(drop=True)
    
    def _validate_dataset_lenient(self, df: pd.DataFrame) -> pd.DataFrame:
        """Lenient validation - just check file existence and basic OpenCV loading"""
        print("🔄 Performing lenient validation (file existence + basic OpenCV)...")
        
        valid_indices = []
        total = len(df)
        
        for idx, row in df.iterrows():
            video_path = row['video_path']
            
            # Lenient criteria: file exists and OpenCV can open it
            if (os.path.exists(video_path) and 
                os.path.getsize(video_path) > 1000 and
                self._basic_opencv_check(video_path)):
                valid_indices.append(idx)
                self.stats['valid'] += 1
            else:
                self.stats['invalid'] += 1
            
            self.stats['total'] += 1
            
            if (idx + 1) % 500 == 0:
                print(f"  Progress: {idx + 1}/{total} files validated...")
        
        return df.iloc[valid_indices].reset_index(drop=True)
    
    def _basic_opencv_check(self, video_path: str) -> bool:
        """Most basic OpenCV check - just try to open"""
        try:
            with ultimate_stderr_suppression():
                cap = cv2.VideoCapture(video_path)
                is_opened = cap.isOpened()
                cap.release()
                return is_opened
        except:
            return False
    
    def print_stats(self):
        """Print comprehensive validation statistics"""
        print(f"\n📊 Final validation statistics:")
        print(f"  Total files: {self.stats['total']}")
        print(f"  ✅ Valid files: {self.stats['valid']}")
        print(f"  ❌ Invalid files: {self.stats['invalid']}")
        print(f"  📁 Not found: {self.stats['not_found']}")
        print(f"  📏 Too small: {self.stats['too_small']}")
        if self.ffprobe_available:
            print(f"  🔧 FFprobe success: {self.stats['ffprobe_success']}")
            print(f"  🔧 FFprobe failed: {self.stats['ffprobe_failed']}")
        print(f"  📹 OpenCV success: {self.stats['opencv_success']}")
        print(f"  📹 OpenCV failed: {self.stats['opencv_failed']}")
        if self.stats['total'] > 0:
            print(f"  📈 Overall success rate: {100 * self.stats['valid'] / self.stats['total']:.1f}%")

# =============================================================================
# 2. ROBUST VIDEO LOADER (SAME AS BEFORE)
# =============================================================================

class RobustVideoLoader:
    """Robust video loader with better error handling"""
    
    def __init__(self, max_frames=8, image_size=224):
        self.max_frames = max_frames
        self.image_size = image_size
        self.video_mean = [0.485, 0.456, 0.406]
        self.video_std = [0.229, 0.224, 0.225]
        self.successful_loads = 0
        self.failed_loads = 0
        
        # Create diverse fallback videos
        self.fallback_videos = self._create_fallback_videos()
    
    def _create_fallback_videos(self):
        """Create multiple types of fallback videos"""
        fallbacks = {}
        
        # Black video
        black = torch.zeros(self.max_frames, 3, self.image_size, self.image_size)
        fallbacks['black'] = self._normalize_video(black)
        
        # Noise video
        noise = torch.randn(self.max_frames, 3, self.image_size, self.image_size) * 0.2
        fallbacks['noise'] = self._normalize_video(noise)
        
        # Simple pattern
        pattern = torch.zeros(self.max_frames, 3, self.image_size, self.image_size)
        for t in range(self.max_frames):
            for c in range(3):
                x = torch.arange(self.image_size).float() / self.image_size
                y = torch.arange(self.image_size).float() / self.image_size
                xx, yy = torch.meshgrid(x, y, indexing='ij')
                pattern[t, c] = (xx + yy + t / self.max_frames) % 1
        fallbacks['pattern'] = self._normalize_video(pattern)
        
        return fallbacks
    
    def _normalize_video(self, video_tensor):
        """Apply ImageNet normalization"""
        mean = torch.tensor(self.video_mean).view(1, 3, 1, 1)
        std = torch.tensor(self.video_std).view(1, 3, 1, 1)
        return (video_tensor - mean) / std
    
    def load_video_robust(self, video_path: str) -> torch.Tensor:
        """Load video with comprehensive error handling"""
        try:
            with ultimate_stderr_suppression():
                cap = None
                
                # Try multiple backends
                for backend in [cv2.CAP_FFMPEG, cv2.CAP_ANY]:
                    try:
                        cap = cv2.VideoCapture(video_path, backend)
                        if cap and cap.isOpened():
                            break
                    except:
                        continue
                
                if not cap or not cap.isOpened():
                    self.failed_loads += 1
                    return self._get_fallback_video()
                
                # Get video properties
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if total_frames <= 0:
                    total_frames = 30  # Assume 30 frames if unknown
                
                # Sample frames
                if total_frames <= self.max_frames:
                    frame_indices = list(range(total_frames)) + [total_frames-1] * (self.max_frames - total_frames)
                else:
                    frame_indices = np.linspace(0, total_frames-1, self.max_frames, dtype=int)
                
                frames = []
                successful_frames = 0
                
                for idx in frame_indices:
                    try:
                        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
                        ret, frame = cap.read()
                        
                        if ret and frame is not None and frame.size > 0:
                            # Process frame
                            frame = cv2.resize(frame, (self.image_size, self.image_size))
                            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                            frame = frame.astype(np.float32) / 255.0
                            
                            # Apply normalization
                            mean = np.array(self.video_mean, dtype=np.float32).reshape(1, 1, 3)
                            std = np.array(self.video_std, dtype=np.float32).reshape(1, 1, 3)
                            frame = (frame - mean) / std
                            
                            frames.append(frame)
                            successful_frames += 1
                        else:
                            # Use previous frame or create dummy
                            if frames:
                                frames.append(frames[-1])
                            else:
                                dummy = np.zeros((self.image_size, self.image_size, 3), dtype=np.float32)
                                mean = np.array(self.video_mean, dtype=np.float32).reshape(1, 1, 3)
                                std = np.array(self.video_std, dtype=np.float32).reshape(1, 1, 3)
                                frames.append((dummy - mean) / std)
                    except:
                        # Fallback frame
                        if frames:
                            frames.append(frames[-1])
                        else:
                            dummy = np.zeros((self.image_size, self.image_size, 3), dtype=np.float32)
                            mean = np.array(self.video_mean, dtype=np.float32).reshape(1, 1, 3)
                            std = np.array(self.video_std, dtype=np.float32).reshape(1, 1, 3)
                            frames.append((dummy - mean) / std)
                
                cap.release()
                
                # Ensure we have enough frames
                while len(frames) < self.max_frames:
                    if frames:
                        frames.append(frames[-1])
                    else:
                        dummy = np.zeros((self.image_size, self.image_size, 3), dtype=np.float32)
                        mean = np.array(self.video_mean, dtype=np.float32).reshape(1, 1, 3)
                        std = np.array(self.video_std, dtype=np.float32).reshape(1, 1, 3)
                        frames.append((dummy - mean) / std)
                
                # Convert to tensor (T, H, W, C) -> (T, C, H, W)
                video_array = np.stack(frames[:self.max_frames])
                video_tensor = torch.from_numpy(video_array).float()
                video_tensor = video_tensor.permute(0, 3, 1, 2)  # (T, C, H, W)
                
                # Accept any video that loads (very lenient)
                self.successful_loads += 1
                return video_tensor
                
        except:
            self.failed_loads += 1
            return self._get_fallback_video()
    
    def _get_fallback_video(self):
        """Get diverse fallback videos"""
        fallback_types = ['black', 'noise', 'pattern']
        fallback_type = fallback_types[self.failed_loads % len(fallback_types)]
        return self.fallback_videos[fallback_type].clone()
    
    def get_stats(self):
        total = self.successful_loads + self.failed_loads
        if total > 0:
            print(f"📹 Video loading: {self.successful_loads}/{total} successful ({100*self.successful_loads/total:.1f}%)")

# =============================================================================
# 3-8. MODEL AND TRAINING CODE (SAME AS BEFORE)
# =============================================================================

@dataclass
class SmartMultimodalConfig(PretrainedConfig):
    model_type = "smart_multimodal"
    
    def __init__(
        self,
        num_classes: int = 4,
        text_model_name: str = "distilbert-base-multilingual-cased",
        video_model_name: str = "facebook/timesformer-base-finetuned-k400",
        text_hidden_size: int = 768,
        video_hidden_size: int = 768,
        fusion_hidden_size: int = 256,
        classifier_hidden_size: int = 128,
        max_frames: int = 8,
        image_size: int = 224,
        dropout: float = 0.1,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.text_model_name = text_model_name
        self.video_model_name = video_model_name
        self.text_hidden_size = text_hidden_size
        self.video_hidden_size = video_hidden_size
        self.fusion_hidden_size = fusion_hidden_size
        self.classifier_hidden_size = classifier_hidden_size
        self.max_frames = max_frames
        self.image_size = image_size
        self.dropout = dropout

class SmartMultimodalModel(PreTrainedModel):
    config_class = SmartMultimodalConfig
    
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        
        print(f"🔤 Initializing text model: {config.text_model_name}")
        self.text_encoder = AutoModel.from_pretrained(config.text_model_name)
        
        print(f"🎬 Initializing video model: {config.video_model_name}")
        self.video_encoder = TimesformerForVideoClassification.from_pretrained(
            config.video_model_name,
            num_labels=config.num_classes,
            ignore_mismatched_sizes=True
        )
        
        # Get actual dimensions
        video_dim = self.video_encoder.config.hidden_size
        text_dim = self.text_encoder.config.hidden_size
        
        self.config.video_hidden_size = video_dim
        self.config.text_hidden_size = text_dim
        
        print(f"📐 Model dimensions - Text: {text_dim}, Video: {video_dim}")
        
        # Projection layers
        self.text_projection = nn.Sequential(
            nn.Linear(text_dim, config.fusion_hidden_size),
            nn.ReLU(),
            nn.Dropout(config.dropout)
        )
        
        self.video_projection = nn.Sequential(
            nn.Linear(video_dim, config.fusion_hidden_size),
            nn.ReLU(),
            nn.Dropout(config.dropout)
        )
        
        # Fusion layers
        self.fusion = nn.Sequential(
            nn.Linear(config.fusion_hidden_size * 2, config.fusion_hidden_size),
            nn.LayerNorm(config.fusion_hidden_size),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.fusion_hidden_size, config.classifier_hidden_size),
            nn.LayerNorm(config.classifier_hidden_size),
            nn.ReLU(),
            nn.Dropout(config.dropout)
        )
        
        self.classifier = nn.Linear(config.classifier_hidden_size, config.num_classes)
        
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"🧠 Model: {total_params:,} total, {trainable_params:,} trainable")
    
    def forward(self, pixel_values, input_ids, attention_mask, labels=None, **kwargs):
        # Text encoding
        text_outputs = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        text_features = text_outputs.last_hidden_state[:, 0, :]
        text_features = self.text_projection(text_features)
        
        # Video encoding with silent error handling
        try:
            # Fix tensor dimensions: (B, T, C, H, W) -> (B, T, C, H, W) (TimeSformer format)
            batch_size, num_frames, channels, height, width = pixel_values.shape
            
            video_outputs = self.video_encoder.timesformer(pixel_values)
            video_features = video_outputs.last_hidden_state[:, 0]  # CLS token
            
        except:
            # Silent fallback
            batch_size = pixel_values.size(0)
            video_features = torch.zeros(
                batch_size, self.config.video_hidden_size,
                dtype=text_features.dtype, device=text_features.device
            )
        
        video_features = self.video_projection(video_features)
        
        # Fusion
        combined_features = torch.cat([text_features, video_features], dim=1)
        fused_features = self.fusion(combined_features)
        logits = self.classifier(fused_features)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(label_smoothing=0.1)
            loss = loss_fct(logits, labels)
        
        return {'loss': loss, 'logits': logits}

class SmartDataManager:
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df = None
        self.classes = None
        self.class_to_id = None
        self.id_to_class = None
        self.num_classes = None
        
    def load_data(self):
        print("📂 Loading dataset...")
        self.df = pd.read_csv(self.csv_path)
        
        # Fix video paths
        # self.df['video_path'] = self.df['video_path'].str.replace(
        #     r'^/kaggle/working/extra_tikharm/',
        #     '/kaggle/input/extra-dataset/',
        #     regex=True
        # )
        
        self.df['combined_text'] = self.df['combined_text'].fillna("")
        
        print(f"Dataset loaded: {len(self.df)} samples")
        print(f"Classes distribution:\n{self.df['class_name'].value_counts()}")
        
        self.classes = sorted(self.df['class_name'].unique())
        self.class_to_id = {cls: i for i, cls in enumerate(self.classes)}
        self.id_to_class = {i: cls for cls, i in self.class_to_id.items()}
        self.num_classes = len(self.classes)
        
        print(f"📊 Classes ({self.num_classes}): {self.classes}")
        
        # Print some sample paths to debug
        print(f"\n🔍 Sample video paths:")
        for i in range(min(5, len(self.df))):
            path = self.df.iloc[i]['video_path']
            exists = os.path.exists(path)
            print(f"  {path} - Exists: {exists}")
        
    def get_smart_splits(self, validator: SmartVideoValidator):
        """Get splits with smart validation"""
        print("🧹 Creating smart validated splits...")
        
        # Get original splits
        # train_df = self.df[self.df['split'] == 'train'].reset_index(drop=True)
        # val_df = self.df[self.df['split'] == 'val'].reset_index(drop=True)
        # test_df = self.df[self.df['split'] == 'test'].reset_index(drop=True)

        train_df = self.df[(self.df['split'] == 'train') & (self.df['original_dir'] != 'extra')].reset_index(drop=True)
        val_df = self.df[(self.df['split'] == 'val') & (self.df['original_dir'] != 'extra')].reset_index(drop=True)
        test_df = self.df[self.df['split'] == 'test'].reset_index(drop=True)
        
        print(f"Original splits - Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
        
        # Use sample validation first
        print("Validating training set...")
        clean_train_df = validator.validate_dataset_sample(train_df, sample_size=100)
        if len(clean_train_df) < 50:
            print("⚠️ Too few training samples, using lenient mode for all...")
            clean_train_df = validator._validate_dataset_lenient(train_df)
        
        print("Validating validation set...")
        clean_val_df = validator.validate_dataset_sample(val_df, sample_size=50)
        if len(clean_val_df) < 10:
            clean_val_df = validator._validate_dataset_lenient(val_df)
        
        print("Validating test set...")
        clean_test_df = validator.validate_dataset_sample(test_df, sample_size=50)
        if len(clean_test_df) < 10:
            clean_test_df = validator._validate_dataset_lenient(test_df)
        
        validator.print_stats()
        
        print(f"\n✨ Final splits:")
        print(f"  🏋️ Train: {len(clean_train_df)} (was {len(train_df)})")
        print(f"  ✅ Val: {len(clean_val_df)} (was {len(val_df)})")
        print(f"  🧪 Test: {len(clean_test_df)} (was {len(test_df)})")
        
        return clean_train_df, clean_val_df, clean_test_df

class SmartDataset(Dataset):
    """Smart dataset with robust video loading AND TRACKING"""
    
    def __init__(self, dataframe, text_tokenizer, max_frames=8, image_size=224, max_text_length=64, dataset_name="unknown"):
        self.df = dataframe
        self.text_tokenizer = text_tokenizer
        self.max_frames = max_frames
        self.image_size = image_size
        self.max_text_length = max_text_length
        self.dataset_name = dataset_name  # Track dataset name
        
        # FIXED: Store actual dataframe length at creation
        self.actual_length = len(self.df)
        
        self.video_loader = RobustVideoLoader(max_frames, image_size)
        
        print(f"🛡️ {self.dataset_name} dataset: {self.actual_length} samples (verified)")
        
        # Store indices for verification
        self.sample_indices = list(self.df.index)
        
    def __len__(self):
        # FIXED: Return actual dataframe length, not processed count
        return self.actual_length
    
    def __getitem__(self, idx):
        try:
            # FIXED: Use actual dataframe index 
            row = self.df.iloc[idx]
            
            # Process text
            text = str(row['combined_text']) if pd.notna(row['combined_text']) else ""
            if not text.strip():
                text = "[EMPTY]"
            
            text_inputs = self.text_tokenizer(
                text,
                max_length=self.max_text_length,
                truncation=True,
                padding='max_length',
                return_tensors='pt'
            )
            
            # Load video robustly
            video_path = row['video_path']
            pixel_values = self.video_loader.load_video_robust(video_path)
            
            # Get label
            label = data_manager.class_to_id[row['class_name']]
            
            return {
                'pixel_values': pixel_values,
                'input_ids': text_inputs['input_ids'].squeeze(0),
                'attention_mask': text_inputs['attention_mask'].squeeze(0),
                'labels': torch.tensor(label, dtype=torch.long)
            }
            
        except Exception as e:
            print(f"⚠️ Error at index {idx} in {self.dataset_name}: {e}")
            
            # Safe fallback
            dummy_video = torch.zeros(self.max_frames, 3, self.image_size, self.image_size)
            dummy_input_ids = torch.zeros(self.max_text_length, dtype=torch.long)
            dummy_attention_mask = torch.zeros(self.max_text_length, dtype=torch.long)
            
            return {
                'pixel_values': dummy_video,
                'input_ids': dummy_input_ids,
                'attention_mask': dummy_attention_mask,
                'labels': torch.tensor(0, dtype=torch.long)
            }
    
    def get_stats(self):
        print(f"📊 {self.dataset_name} stats:")
        print(f"  DataFrame length: {len(self.df)}")
        print(f"  Actual length: {self.actual_length}")
        self.video_loader.get_stats()

def compute_comprehensive_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
    """Compute all 4 key metrics"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    labels = eval_pred.label_ids
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    recall = recall_score(labels, predictions, average='macro', zero_division=0)
    precision = precision_score(labels, predictions, average='macro', zero_division=0)
    
    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'recall': recall,
        'precision': precision
    }

def smart_collate_fn(batch):
    """Smart collate function"""
    try:
        return {
            'pixel_values': torch.stack([item['pixel_values'] for item in batch]),
            'input_ids': torch.stack([item['input_ids'] for item in batch]),
            'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
            'labels': torch.stack([item['labels'] for item in batch])
        }
    except Exception as e:
        print(f"⚠️ Collate error: {e}")
        batch_size = len(batch)
        return {
            'pixel_values': torch.zeros(batch_size, 8, 3, 224, 224),
            'input_ids': torch.zeros(batch_size, 64, dtype=torch.long),
            'attention_mask': torch.zeros(batch_size, 64, dtype=torch.long),
            'labels': torch.zeros(batch_size, dtype=torch.long)
        }

# =============================================================================
# MAIN EXECUTION WITH FIXED DATA TRACKING
# =============================================================================

def main():
    """Main training function with FIXED data counting"""
    
    try:
        print("🚀 Starting smart adaptive training (FIXED VERSION)...")
        
        # Initialize data manager
        global data_manager
        data_manager = SmartDataManager('/kaggle/input/processed-data/tikharm_vi_en_complete.csv')
        data_manager.load_data()
        
        # Initialize smart validator (not strict)
        print("🔍 Initializing smart video validator...")
        validator = SmartVideoValidator(strict_mode=False)
        
        # Get smart validated data
        train_df, val_df, test_df = data_manager.get_smart_splits(validator)
        
        # VERIFY DATA SPLITS AGAIN
        print(f"\n🔍 VERIFICATION - Actual dataframe sizes:")
        print(f"  Train DF: {len(train_df)} rows")
        print(f"  Val DF: {len(val_df)} rows") 
        print(f"  Test DF: {len(test_df)} rows")
        
        # Check data sufficiency
        if len(train_df) < 50:
            print("⚠️ Very few training samples!")
            print("🔄 Proceeding with available data...")
        
        # Initialize models
        print("🧠 Initializing models...")
        text_model_name = "distilbert-base-multilingual-cased"
        text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
        
        config = SmartMultimodalConfig(
            num_classes=data_manager.num_classes,
            text_model_name=text_model_name,
            max_frames=8,
            image_size=224,
            dropout=0.1
        )
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"💻 Using device: {device}")
        
        model = SmartMultimodalModel(config)
        model.to(device)
        
        # Create smart datasets WITH PROPER TRACKING
        print("📊 Creating smart datasets with tracking...")
        train_dataset = SmartDataset(train_df, text_tokenizer, dataset_name="TRAIN")
        val_dataset = SmartDataset(val_df, text_tokenizer, dataset_name="VAL")
        test_dataset = SmartDataset(test_df, text_tokenizer, dataset_name="TEST")
        
        # VERIFY DATASET LENGTHS
        print(f"\n🎯 FINAL DATASET VERIFICATION:")
        print(f"  Train dataset: {len(train_dataset)} samples")
        print(f"  Val dataset: {len(val_dataset)} samples")
        print(f"  Test dataset: {len(test_dataset)} samples")
        
        # Test sample loading
        print("🧪 Testing sample loading...")
        sample = train_dataset[0]
        print(f"  ✅ Video shape: {sample['pixel_values'].shape}")
        print(f"  ✅ Text shape: {sample['input_ids'].shape}")
        
        # Test forward pass
        model.eval()
        with torch.no_grad():
            batch = {k: v.unsqueeze(0).to(device) for k, v in sample.items()}
            output = model(**batch)
            print(f"  ✅ Output shape: {output['logits'].shape}")
        
        # Training setup with FIXED batch settings
        training_args = TrainingArguments(
            output_dir='./smart-multimodal-fixed',
            num_train_epochs=6,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            gradient_accumulation_steps=2,
            learning_rate=1e-4,
            weight_decay=0.01,
            warmup_steps=100,
            logging_steps=50,
            eval_strategy="steps",
            eval_steps=50,
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model="f1_score",
            fp16=True,
            dataloader_num_workers=0,
            remove_unused_columns=False,
            report_to=None,
            save_total_limit=2,
            # FIXED: Don't drop last to preserve data count
            dataloader_drop_last=False,  # Changed from True to False
            max_grad_norm=1.0,
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            data_collator=smart_collate_fn,
            compute_metrics=compute_comprehensive_metrics,
            tokenizer=text_tokenizer,
        )
        
        # Start training
        print("\n" + "="*80)
        print("🚀 STARTING SMART ADAPTIVE TRAINING (FIXED)!")
        print("="*80)
        
        start_time = time.time()
        
        # Train with error suppression
        with ultimate_stderr_suppression():
            train_result = trainer.train()
        
        training_time = time.time() - start_time
        
        print(f"\n🎉 TRAINING COMPLETED in {training_time:.1f}s!")
        
        # Save model
        trainer.save_model("./final-smart-model-fixed")
        print("💾 Model saved!")
        
        # Evaluation
        print("📊 Running evaluation...")
        eval_results = trainer.evaluate()
        
        print(f"\n🎯 VALIDATION RESULTS:")
        print(f"  Accuracy: {eval_results['eval_accuracy']:.4f} ({eval_results['eval_accuracy']*100:.2f}%)")
        print(f"  F1-Score: {eval_results['eval_f1_score']:.4f}")
        print(f"  Recall: {eval_results['eval_recall']:.4f}")
        print(f"  Precision: {eval_results['eval_precision']:.4f}")
        
        # FIXED Test evaluation with manual prediction
        print("🧪 Running FIXED test evaluation...")
        
        # Use manual DataLoader to control exact sample count
        test_dataloader = DataLoader(
            test_dataset, 
            batch_size=8, 
            shuffle=False, 
            drop_last=False,  # Don't drop any samples
            collate_fn=smart_collate_fn
        )
        
        model.eval()
        all_predictions = []
        all_labels = []
        
        print(f"🔍 Processing test data in {len(test_dataloader)} batches...")
        
        with torch.no_grad():
            for batch_idx, batch in enumerate(test_dataloader):
                # Move to device
                batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}
                
                # Get predictions
                outputs = model(**batch)
                predictions = torch.argmax(outputs['logits'], dim=1)
                
                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(batch['labels'].cpu().numpy())
                
                if (batch_idx + 1) % 10 == 0:
                    print(f"  Processed batch {batch_idx + 1}/{len(test_dataloader)}")
        
        # Convert to numpy arrays
        y_pred = np.array(all_predictions)
        y_true = np.array(all_labels)
        
        print(f"\n🎯 FINAL TEST DATA VERIFICATION:")
        print(f"  Expected test samples: {len(test_dataset)}")
        print(f"  Actual predictions: {len(y_pred)}")
        print(f"  Actual labels: {len(y_true)}")
        print(f"  Data consistency: {'✅ CONSISTENT' if len(y_pred) == len(test_dataset) else '❌ INCONSISTENT'}")
        
        # Test metrics
        test_accuracy = accuracy_score(y_true, y_pred)
        test_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
        test_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
        test_precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
        
        print(f"\n🏆 FINAL TEST RESULTS (VERIFIED):")
        print(f"  Test samples: {len(y_true)}")
        print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
        print(f"  F1-Score: {test_f1:.4f}")
        print(f"  Recall: {test_recall:.4f}")
        print(f"  Precision: {test_precision:.4f}")
        
        # Classification report
        print(f"\n📋 CLASSIFICATION REPORT (VERIFIED):")
        print("-" * 60)
        class_report = classification_report(
            y_true, y_pred, 
            target_names=data_manager.classes,
            digits=4
        )
        print(class_report)
        
        # Save results
        import json
        results = {
            'model_type': 'Smart_Adaptive_Multimodal_FIXED',
            'data_verification': {
                'train_samples': len(train_dataset),
                'val_samples': len(val_dataset),
                'test_samples': len(test_dataset),
                'test_predictions': len(y_pred),
                'data_consistent': len(y_pred) == len(test_dataset)
            },
            'validation_metrics': {
                'accuracy': eval_results['eval_accuracy'],
                'f1_score': eval_results['eval_f1_score'],
                'recall': eval_results['eval_recall'],
                'precision': eval_results['eval_precision']
            },
            'test_metrics': {
                'accuracy': test_accuracy,
                'f1_score': test_f1,
                'recall': test_recall,
                'precision': test_precision
            },
            'training_time_seconds': training_time,
            'validation_stats': validator.stats,
            'dataset_sizes': {
                'train': len(train_dataset),
                'val': len(val_dataset),
                'test': len(test_dataset)
            }
        }
        
        with open('smart_results_fixed.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        print("💾 Results saved!")
        
        # Final statistics
        print("\n📈 Final statistics:")
        train_dataset.get_stats()
        val_dataset.get_stats() 
        test_dataset.get_stats()
        
        print("\n" + "="*80)
        print("🎊 SMART TRAINING COMPLETED SUCCESSFULLY (FIXED)!")
        print("✅ DATA COUNTING ISSUES RESOLVED!")
        print("📊 ALL METRICS CALCULATED CORRECTLY!")
        print("="*80)
        
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        cleanup_memory()

# Run the main function
if __name__ == "__main__":
    main()

2025-07-01 16:16:37.580290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751386597.772844      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751386597.828632      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


=== SMART MULTIMODAL CLASSIFIER - ADAPTIVE VALIDATION (FIXED) ===
🚀 Starting smart adaptive training (FIXED VERSION)...
📂 Loading dataset...
Dataset loaded: 4723 samples
Classes distribution:
class_name
Safe               1248
Harmful Content    1193
Adult Content      1141
Suicide            1141
Name: count, dtype: int64
📊 Classes (4): ['Adult Content', 'Harmful Content', 'Safe', 'Suicide']

🔍 Sample video paths:
  /kaggle/input/tikharm-dataset/Dataset/train/Adult Content/jaypark_foryou_7339639860354911493.mp4 - Exists: True
  /kaggle/input/tikharm-dataset/Dataset/train/Adult Content/1life.anthonyblane_7368846422709521671.mp4 - Exists: True
  /kaggle/input/tikharm-dataset/Dataset/train/Adult Content/loungebarvip_7252268249243454725.mp4 - Exists: True
  /kaggle/input/tikharm-dataset/Dataset/train/Adult Content/phangphong111_7287936860796620039.mp4 - Exists: True
  /kaggle/input/tikharm-dataset/Dataset/train/Adult Content/girl_xinh.sexy_7298717939052711170.mp4 - Exists: True
🔍 Initiali

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


  ✅ Output shape: torch.Size([1, 4])

🚀 STARTING SMART ADAPTIVE TRAINING (FIXED)!


Step,Training Loss,Validation Loss,Accuracy,F1 Score,Recall,Precision
50,2.377300,0.832425,0.777778,0.775290,0.777872,0.786832
100,1.564100,0.733158,0.800505,0.800105,0.800193,0.817634
150,1.474300,0.675323,0.848485,0.846743,0.848326,0.855256
200,1.330100,0.693130,0.838384,0.837591,0.838352,0.850635
250,1.258900,0.706666,0.848485,0.846578,0.848482,0.860118
300,1.199400,0.738048,0.833333,0.833003,0.833810,0.843735
350,1.195300,0.730099,0.840909,0.842149,0.841183,0.854065
400,1.042700,0.705074,0.853535,0.854745,0.853400,0.858301
450,1.032700,0.705230,0.845960,0.846958,0.845568,0.859192
500,0.986300,0.682787,0.863636,0.863010,0.863656,0.870290



🎉 TRAINING COMPLETED in 25493.7s!
💾 Model saved!
📊 Running evaluation...



🎯 VALIDATION RESULTS:
  Accuracy: 0.8864 (88.64%)
  F1-Score: 0.8856
  Recall: 0.8862
  Precision: 0.8862
🧪 Running FIXED test evaluation...
🔍 Processing test data in 99 batches...
  Processed batch 10/99
  Processed batch 20/99
  Processed batch 30/99
  Processed batch 40/99
  Processed batch 50/99
  Processed batch 60/99
  Processed batch 70/99
  Processed batch 80/99
  Processed batch 90/99

🎯 FINAL TEST DATA VERIFICATION:
  Expected test samples: 790
  Actual predictions: 790
  Actual labels: 790
  Data consistency: ✅ CONSISTENT

🏆 FINAL TEST RESULTS (VERIFIED):
  Test samples: 790
  Accuracy: 0.8937 (89.37%)
  F1-Score: 0.8936
  Recall: 0.8936
  Precision: 0.8945

📋 CLASSIFICATION REPORT (VERIFIED):
------------------------------------------------------------
                 precision    recall  f1-score   support

  Adult Content     0.9149    0.8821    0.8982       195
Harmful Content     0.8750    0.8485    0.8615       198
           Safe     0.9289    0.9150    0.9219      

In [4]:
# import pandas as pd
# import torch
# import torch.nn as nn
# import numpy as np
# import os
# import cv2
# from torch.utils.data import Dataset, DataLoader
# from transformers import (
#     AutoTokenizer, AutoModel, AutoImageProcessor,
#     TimesformerForVideoClassification, TimesformerConfig,
#     TrainingArguments, Trainer
# )
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns
# import gc
# import warnings
# warnings.filterwarnings('ignore')

# df = pd.read_csv('/kaggle/input/processed-data/tikharm_vi_en_complete.csv')
# print(f"Dataset shape: {df.shape}")
# print(f"Classes: {df['class_name'].value_counts()}")
# print(f"Splits: {df['split'].value_counts()}")

# df['combined_text'] = df['combined_text'].fillna("")

# classes = sorted(df['class_name'].unique())
# label2id = {label: i for i, label in enumerate(classes)}
# id2label = {i: label for i, label in label2id.items()}
# num_classes = len(classes)

# print(f"Number of classes: {num_classes}")
# print(f"Classes: {classes}")

# train_df = df[df['split'] == 'train'].reset_index(drop=True)
# val_df = df[df['split'] == 'val'].reset_index(drop=True)
# test_df = df[df['split'] == 'test'].reset_index(drop=True)

# print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# class TimeSformerMultimodalModel(nn.Module):
#     def __init__(self, num_classes, 
#                  text_model_name="distilbert-base-multilingual-cased",
#                  video_model_name="facebook/timesformer-base-finetuned-k400"):
#         super().__init__()
        
#         self.text_encoder = AutoModel.from_pretrained(text_model_name)
#         text_dim = self.text_encoder.config.hidden_size
        
#         self.timesformer_config = TimesformerConfig(
#             image_size=112,                    # Giảm từ 224 -> 112 để tiết kiệm memory
#             patch_size=16,                     # Giữ nguyên
#             num_channels=3,
#             num_frames=8,                      # Tăng từ 4 -> 8 frames để có thông tin tốt hơn
#             hidden_size=384,                   # Giảm từ 768 -> 384 để tiết kiệm memory
#             num_hidden_layers=6,               # Giảm từ 12 -> 6 layers
#             num_attention_heads=6,             # Giảm từ 12 -> 6 heads
#             intermediate_size=1536,            # Giảm từ 3072 -> 1536
#             hidden_dropout_prob=0.1,
#             attention_probs_dropout_prob=0.1,
#             attention_type="divided_space_time", # Tối ưu nhất theo paper
#             drop_path_rate=0.1,
#             num_labels=num_classes,
#             qkv_bias=True
#         )
        
#         print("Loading TimeSformer...")
#         try:
#             self.video_encoder = TimesformerForVideoClassification(self.timesformer_config)
            
#             pretrained_model = TimesformerForVideoClassification.from_pretrained(
#                 video_model_name, 
#                 ignore_mismatched_sizes=True
#             )
            
#             self._transfer_weights(pretrained_model, self.video_encoder)
            
#         except Exception as e:
#             print(f"Warning: Could not load pretrained weights: {e}")
#             self.video_encoder = TimesformerForVideoClassification(self.timesformer_config)
        
#         video_dim = self.timesformer_config.hidden_size
        
#         self.fusion = nn.Sequential(
#             nn.Linear(text_dim + video_dim, 256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 128),
#             nn.ReLU(), 
#             nn.Dropout(0.2),
#             nn.Linear(128, num_classes)
#         )
        
#     def _transfer_weights(self, source_model, target_model):
#         source_dict = source_model.state_dict()
#         target_dict = target_model.state_dict()
        
#         transferred = 0
#         for name, param in target_dict.items():
#             if name in source_dict and source_dict[name].shape == param.shape:
#                 target_dict[name] = source_dict[name]
#                 transferred += 1
        
#         target_model.load_state_dict(target_dict)
#         print(f"Transferred {transferred}/{len(target_dict)} layers from pretrained model")
        
#     def forward(self, input_ids, attention_mask, pixel_values, labels=None):
#         text_output = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
#         text_features = text_output.last_hidden_state.mean(dim=1)
        
#         video_output = self.video_encoder.timesformer(pixel_values)
        
#         video_features = video_output.last_hidden_state[:, 0]  # [batch_size, hidden_size]
        
#         combined = torch.cat([text_features, video_features], dim=1)
#         logits = self.fusion(combined)
        
#         loss = None
#         if labels is not None:
#             loss = nn.CrossEntropyLoss()(logits, labels)
            
#         return {'loss': loss, 'logits': logits}


# def timesformer_video_loader(video_path, target_frames=8, img_size=112):
#     """Load video compatible với TimeSformer"""
#     try:
#         cap = cv2.VideoCapture(video_path)
#         frames = []
        
#         total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#         if total_frames == 0:
#             cap.release()
#             return torch.zeros(3, target_frames, img_size, img_size)
            
#         # Sample frames uniformly
#         if total_frames < target_frames:
#             # Repeat frames if video is too short
#             indices = np.linspace(0, max(0, total_frames-1), target_frames, dtype=int)
#         else:
#             # Sample uniformly
#             indices = np.linspace(0, total_frames-1, target_frames, dtype=int)
        
#         for idx in indices:
#             cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
#             ret, frame = cap.read()
#             if ret:
#                 # Resize và normalize như TimeSformer expects
#                 frame = cv2.resize(frame, (img_size, img_size))
#                 frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 frame = frame.astype(np.float32) / 255.0
#                 frames.append(frame)
#             else:
#                 # Dummy frame
#                 frames.append(np.zeros((img_size, img_size, 3), dtype=np.float32))
                
#         cap.release()
        
#         # Convert to tensor: (T, H, W, C) -> (C, T, H, W)
#         if len(frames) == target_frames:
#             video_tensor = torch.from_numpy(np.array(frames))
#             video_tensor = video_tensor.permute(3, 0, 1, 2)
#             return video_tensor
#         else:
#             return torch.zeros(3, target_frames, img_size, img_size)
            
#     except Exception as e:
#         return torch.zeros(3, target_frames, img_size, img_size)

# class TimeSformerDataset(Dataset):
#     def __init__(self, dataframe, tokenizer, image_processor=None, 
#                  max_length=64, img_size=112, num_frames=8):
#         self.df = dataframe
#         self.tokenizer = tokenizer
#         self.image_processor = image_processor
#         self.max_length = max_length
#         self.img_size = img_size
#         self.num_frames = num_frames
        
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
        
#         # Text
#         text = str(row['combined_text']) if pd.notna(row['combined_text']) else ""
#         text_inputs = self.tokenizer(
#             text,
#             max_length=self.max_length,
#             padding='max_length', 
#             truncation=True,
#             return_tensors='pt'
#         )
        
#         # Video
#         video_path = row['video_path']
#         if pd.notna(video_path) and os.path.exists(video_path):
#             pixel_values = timesformer_video_loader(video_path, self.num_frames, self.img_size)
#         else:
#             pixel_values = torch.zeros(3, self.num_frames, self.img_size, self.img_size)
        
#         if self.image_processor is not None:
#             try:
#                 video_np = pixel_values.permute(1, 2, 3, 0).numpy()  # (T, H, W, C)
#                 video_list = [video_np[i] for i in range(self.num_frames)]
                
#                 processed = self.image_processor(video_list, return_tensors="pt")
#                 pixel_values = processed['pixel_values'].squeeze(0)  # Remove batch dim
#             except:
#                 mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1, 1)
#                 std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1, 1)
#                 pixel_values = (pixel_values - mean) / std
        
#         # Label
#         label = label2id[row['class_name']]
        
#         return {
#             'input_ids': text_inputs['input_ids'].squeeze(0),
#             'attention_mask': text_inputs['attention_mask'].squeeze(0),
#             'pixel_values': pixel_values,
#             'labels': torch.tensor(label, dtype=torch.long)
#         }

# print("Initializing models...")

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

# try:
#     image_processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
#     print("Loaded TimeSformer image processor")
# except:
#     image_processor = None
#     print("Could not load image processor, using manual normalization")

# model = TimeSformerMultimodalModel(num_classes=num_classes)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# print(f"Model loaded on: {device}")
# print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# print("Creating datasets...")
# train_dataset = TimeSformerDataset(
#     train_df, tokenizer, image_processor,
#     max_length=64, img_size=112, num_frames=8
# )

# val_dataset = TimeSformerDataset(
#     val_df, tokenizer, image_processor,
#     max_length=64, img_size=112, num_frames=8
# )

# test_dataset = TimeSformerDataset(
#     test_df, tokenizer, image_processor,
#     max_length=64, img_size=112, num_frames=8
# )

# print(f"Train samples: {len(train_dataset)}")
# print(f"Val samples: {len(val_dataset)}")
# print(f"Test samples: {len(test_dataset)}")

# def collate_fn(batch):
#     try:
#         return {
#             'input_ids': torch.stack([item['input_ids'] for item in batch]),
#             'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
#             'pixel_values': torch.stack([item['pixel_values'] for item in batch]),
#             'labels': torch.stack([item['labels'] for item in batch])
#         }
#     except Exception as e:
#         print(f"Collate error: {e}")
#         batch_size = len(batch)
#         return {
#             'input_ids': torch.zeros(batch_size, 64, dtype=torch.long),
#             'attention_mask': torch.zeros(batch_size, 64, dtype=torch.long),
#             'pixel_values': torch.zeros(batch_size, 3, 8, 112, 112),
#             'labels': torch.zeros(batch_size, dtype=torch.long)
#         }

# def compute_metrics(eval_pred):
#     predictions = np.argmax(eval_pred.predictions, axis=1)
#     return {'accuracy': accuracy_score(eval_pred.label_ids, predictions)}

# # Training arguments
# training_args = TrainingArguments(
#     output_dir="./timesformer-multimodal",
#     num_train_epochs=5,
#     per_device_train_batch_size=2,    # Small batch for TimeSformer
#     per_device_eval_batch_size=2,
#     gradient_accumulation_steps=8,    # Effective batch size = 16
#     learning_rate=1e-5,               # Lower LR for pretrained TimeSformer
#     weight_decay=0.01,
#     warmup_steps=200,
#     logging_steps=50,                 # Log mỗi 50 steps
#     eval_strategy="steps",            # Thay đổi từ "epoch" -> "steps"
#     eval_steps=50,                    # Đánh giá mỗi 50 steps
#     save_strategy="steps",            # Thay đổi từ "epoch" -> "steps"
#     save_steps=50,                    # Save mỗi 50 steps
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     fp16=True,
#     dataloader_num_workers=0,
#     remove_unused_columns=False,
#     report_to="none",
#     save_total_limit=3,               # Tăng từ 1 -> 3 để lưu nhiều checkpoint hơn
#     max_grad_norm=1.0,
#     dataloader_drop_last=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=collate_fn,
#     compute_metrics=compute_metrics,
# )

# try:
#     sample = train_dataset[0]
#     print("Sample shapes:")
#     print(f"  Input IDs: {sample['input_ids'].shape}")
#     print(f"  Pixel values: {sample['pixel_values'].shape}")
#     print(f"  Label: {sample['labels']}")
    
#     # Test forward pass
#     model.eval()
#     with torch.no_grad():
#         batch = {k: v.unsqueeze(0).to(device) for k, v in sample.items()}
#         output = model(**batch)
#         print(f"  Output logits shape: {output['logits'].shape}")
    
#     print("Sample loading and forward pass successful!")
# except Exception as e:
#     print(f"Sample test failed: {e}")
#     import traceback
#     traceback.print_exc()

# gc.collect()
# torch.cuda.empty_cache() if torch.cuda.is_available() else None

# training_successful = False
# try:
#     print("Training with TimeSformer...")
#     train_result = trainer.train()
#     print("Training completed!")
    
#     # Save model
#     trainer.save_model("./final-timesformer-multimodal")
#     print("Model saved!")
    
#     # Validation evaluation
#     eval_results = trainer.evaluate()
#     print(f"Validation accuracy: {eval_results['eval_accuracy']:.4f}")
#     print(f"Validation loss: {eval_results['eval_loss']:.4f}")
    
#     training_successful = True
    
# except Exception as e:
#     print(f"Training error: {e}")
#     import traceback
#     traceback.print_exc()

# print("\n" + "=" * 60)
# print("TIMESFORMER MULTIMODAL TEST EVALUATION")
# print("=" * 60)

# if training_successful:
#     try:
#         print("Predicting on full test set...")
#         test_results = trainer.predict(test_dataset)
        
#         y_pred = np.argmax(test_results.predictions, axis=1)
#         y_true = test_results.label_ids
        
#         test_accuracy = accuracy_score(y_true, y_pred)
#         print(f"\nTest Accuracy: {test_accuracy:.4f}")
#         print(f"Test Loss: {test_results.metrics['test_loss']:.4f}")
        
#         # Classification report
#         print("\nClassification Report:")
#         print("-" * 60)
#         class_report = classification_report(
#             y_true, y_pred, 
#             target_names=classes,
#             digits=4
#         )
#         print(class_report)
        
#         # Confusion Matrix
#         cm = confusion_matrix(y_true, y_pred)
#         print("\nConfusion Matrix:")
#         print("-" * 40)
#         header = "".join([f"{cls[:10]:>10}" for cls in classes])
#         print(f"{'':>15}{header}")
        
#         for i, actual_class in enumerate(classes):
#             row = "".join([f"{cm[i,j]:>10}" for j in range(len(classes))])
#             print(f"{actual_class[:13]:>13}: {row}")
        
#         # Performance summary
#         print("\n" + "=" * 50)
#         print("TIMESFORMER MULTIMODAL PERFORMANCE")
#         print("=" * 50)
#         print(f"Architecture: TimeSformer + DistilBERT")
#         print(f"Video frames: 8")
#         print(f"Image size: 112x112")
#         print(f"Test Accuracy: {test_accuracy:.4f}")
#         print(f"Model Parameters: {sum(p.numel() for p in model.parameters()):,}")
        
#         # Save results
#         import json
#         results = {
#             'model_type': 'TimeSformer_Multimodal',
#             'test_accuracy': test_accuracy,
#             'test_loss': test_results.metrics['test_loss'],
#             'classification_report': class_report,
#             'confusion_matrix': cm.tolist(),
#             'classes': classes,
#             'model_config': {
#                 'video_frames': 8,
#                 'image_size': 112,
#                 'hidden_size': 384,
#                 'num_layers': 6,
#                 'attention_heads': 6
#             }
#         }
        
#         with open('timesformer_results.json', 'w') as f:
#             json.dump(results, f, indent=2)
        
#         print("Results saved to timesformer_results.json")
        
#     except Exception as e:
#         print(f"Testing error: {e}")
#         import traceback
#         traceback.print_exc()

# print("\n" + "=" * 60)
# print("TIMESFORMER EXPERIMENT COMPLETE!")
# print("=" * 60)

# gc.collect()
# torch.cuda.empty_cache() if torch.cuda.is_available() else None